In [ ]:
# Packages we need for all the excercises in this notebook
%pylab inline
from __future__ import print_function
import pandas as pd
import numpy as np
import scipy
import sklearn
import matplotlib.pyplot as plt
import psycopg2
import sqlalchemy
import jellyfish
import re
import recordlinkage as rl

In [ ]:
# Read in data needed from database
# db_name = "appliedda"
# db_host = "10.10.2.10"
# pgsql_connection = psycopg2.connect( host = db_host, database = db_name )
# cur = pgsql_connection.cursor()
pgsql_engine = sqlalchemy.create_engine( "postgresql://10.10.2.10/appliedda" )

In [ ]:
exitVars="docnbr, birthyr,race,sex,curadmdt,curadmmo,curadmyr,admtypo,relinst,lstsclvl, hclass,hofnscd,exittyp,ACTMSRDT, ACTMSRMO, ACTMSRYR, ACTDISDT, ACTDISMO, ACTDISYR, CCCADMDT, CCCADMMO, CCCADMYR, JAILTIME, MARSTAT, KIDS, EDUCLVL, GANG, VETF, DRUGALCF, DRUGCOCF, DRUGAMPF, DRUGMARF, DRUGHERF, DRUGPCPF, DRUGOTHF, DRUGUNKF, EMPPLANF, GANGACT, BIRTHPL, CITZSHP, ZIPCODE, SEXREG, SEXOFF, EXITDA, EXITMO, EXITYR, BIRTH_YEAR, BIRTH_DATE, CURADM_DATE, ACTDIS_DATE, EXIT_DATE, ZIP5,kids,marstat,otincar,gttyp01,gttyp02, gttyp03, gttyp04, gttyp05, gttyp06, gttyp07, gttyp08, gttyp09, gttyp10, gttyp11, gttyp12, gttyp13, gttyp14, gttyp15, gttyp16, gttyp17, gttyp18, gttyp19,gttyp24"


In [ ]:
# Select variables needed from ildoc.ildoc_exit
query = "SELECT {Vars} FROM {table} where (exityr between 2010 and 2013) and exit_date-to_date(curadmyr||'-'||curadmmo||'-'||curadmda,'yyyy-mm-dd')!=0;".format(table="ildoc.ildoc_exit", Vars=exitVars)

In [ ]:
#load exit data (exits 2010-2013)
df_ext = pd.read_sql( query, con = pgsql_engine )

In [ ]:
print(df_ext.head())

In [ ]:
#cut months in exit data into quarters
labels= [1, 2, 3, 4]
df_ext['exitqtr']=pd.cut(df_ext.exitmo, range(0, 13, 3), labels=labels)
print(df_ext.head(5))

In [ ]:
#Create variable in exit dataset that identifies 8th quarter and year after release
#Find wage entries for each person that equals or is greater than exit date and equals or is less than 8th quarter
df_ext['yearplus2']=df_ext['exityr'] + 2
df_ext.head(5)

In [ ]:
##Create start=exit year and quarter and end=yearplus2 and quarter
df_ext['start'] = (10 * df_ext['exityr'] ) + df_ext['exitqtr']
df_ext['end'] = (10 * df_ext['yearplus2'] ) + df_ext['exitqtr']
df_ext.head(10)

In [ ]:
#create variables with date for each quarter (year + quarter)
#then, can match dates to wage dates
#if quarter = 4, year+1 and quarter=1
#if df_ext['']
#df_ext['q1'] = (10 * df_ext['exityr'] ) + df_ext['exitqtr']

##start to q0
df_ext['q0']= df_ext['start']
df_ext['q0'] = df_ext['q0'].apply(int)

##Create q1 variable
df_ext['q1']= df_ext['start'] + 1
df_ext['q1'] = df_ext['q1'].apply(int)
df_ext.ix[df_ext.q1 % 5 == 0, 'q1'] += 6

##Create q2 variable
df_ext['q2']= df_ext['q1'] + 1
df_ext['q2'] = df_ext['q2'].apply(int)
df_ext.ix[df_ext.q2 % 5 == 0, 'q2'] += 6

##Create q3 variable
df_ext['q3']= df_ext['q2'] + 1
df_ext['q3'] = df_ext['q3'].apply(int)
df_ext.ix[df_ext.q3 % 5 == 0, 'q3'] += 6

##Create q4 variable
df_ext['q4']= df_ext['q3'] + 1
df_ext['q4'] = df_ext['q4'].apply(int)
df_ext.ix[df_ext.q4 % 5 == 0, 'q4'] += 6

##Create q5 variable
df_ext['q5']= df_ext['q4'] + 1
df_ext['q5'] = df_ext['q5'].apply(int)
df_ext.ix[df_ext.q5 % 5 == 0, 'q5'] += 6

##Create q6 variable
df_ext['q6']= df_ext['q5'] + 1
df_ext['q6'] = df_ext['q6'].apply(int)
df_ext.ix[df_ext.q6 % 5 == 0, 'q6'] += 6

##end to q7
df_ext['q7']=  df_ext['q6'] + 1
df_ext['q7'] = df_ext['q7'].apply(int)
df_ext.ix[df_ext.q7 % 5 == 0, 'q7'] += 6

In [ ]:
df_ext.head(5)

In [ ]:
#send the new table back to pgAdmin
df_ext.to_sql( "exitqtr", con = pgsql_engine, schema = 'm3' )

In [ ]:
#check to make sure the table was created properly
df_exqtr= pd.read_sql( 'SELECT * FROM m3.exitqtr', con = pgsql_engine )
df_exqtr.head(5)